In [2]:
! ls

analysis.ipynb
anli_none_dev_r1_bigscience.T0
anli_none_dev_r1_bigscience.T0_3B
anli_none_dev_r2_bigscience.T0
anli_none_dev_r2_bigscience.T0_3B
anli_none_dev_r3_bigscience.T0
anli_none_dev_r3_bigscience.T0_3B
hellaswag_none_validation_bigscience.T0
hellaswag_none_validation_bigscience.T0_3B
super_glue_cb_validation_bigscience.T0
super_glue_cb_validation_bigscience.T0_3B
super_glue_copa_validation_bigscience.T0
super_glue_copa_validation_bigscience.T0_3B
super_glue_rte_validation_bigscience.T0
super_glue_rte_validation_bigscience.T0_3B
super_glue_wic_validation_bigscience.T0
super_glue_wic_validation_bigscience.T0_3B
super_glue_wsc.fixed_validation_bigscience.T0
super_glue_wsc.fixed_validation_bigscience.T0_3B
winogrande_winogrande_xl_validation_bigscience.T0
winogrande_winogrande_xl_validation_bigscience.T0_3B


In [3]:
from promptsource.templates import DatasetTemplates
import datasets
data = datasets.load_dataset("anli")
data = data["dev_r1"]
prompts = DatasetTemplates("anli", None)
prompt_names = prompts.all_template_names
print(prompt_names)
pname = 'guaranteed/possible/impossible'
k = 1
input, output = prompts[pname].apply(data[k])
targets = prompts[pname].get_answer_choices_list(data[k])

print(output.strip())
print("========")
print(data[k])
print("===========")
print(input)
print("========")
print(output)
print()
print(targets)

Reusing dataset anli (/data/home/chuntinz/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)


  0%|          | 0/9 [00:00<?, ?it/s]

['GPT-3 style', 'MNLI crowdsource', 'always/sometimes/never', 'based on the previous passage', 'can we infer', 'claim true/false/inconclusive', 'consider always/sometimes/never', 'does it follow that', 'does this imply', 'guaranteed true', 'guaranteed/possible/impossible', 'justified in saying', 'must be true', 'should assume', 'take the following as truth']
Guaranteed
{'uid': 'e66af435-eef1-491b-af2a-4825d54611e1', 'premise': 'Franco Zeffirelli, KBE Grande Ufficiale OMRI (] ; born 12 February 1923) is an Italian director and producer of operas, films and television. He is also a former senator (1994–2001) for the Italian centre-right "Forza Italia" party. Recently, Italian researchers have found that he is one of the few distant relatives of Leonardo da Vinci.', 'hypothesis': 'Franco Zeffirelli had a political career', 'label': 0, 'reason': 'Franco Zeffirelli was a senator so he had a political career. The system likely was fooled because I used words not used in the context.'}
Assume

In [2]:
import numpy as np
from collections import Counter
import scipy.stats

def read_file(fname):
    lidx = -1
    eidx = -1
    all_preds_per_example = []
    all_golds = []
    all_preds_per_prompt = []
    with open(fname) as fin:
        for line in fin:
            lidx += 1
            if lidx == 0:
                fields = line.strip().split(',')
                num_prompts, num_examples = int(fields[0].split("=")[1].strip()), int(fields[1].split("=")[1].strip())
                all_preds_per_prompt = [[] for _ in range(num_prompts)]
            elif lidx == 1:
                fields = line.strip().split(',')
                max_acc, median_acc, min_acc, ensemble_acc = float(fields[0].split("=")[1].strip()), float(fields[1].split("=")[1].strip()), float(fields[3].split("=")[1].strip()), float(fields[-1].split("=")[1].strip())
            elif lidx == 2:
                prompts_accuracy = list(map(float, line.strip().split(':')[1].strip().split()))
            elif lidx == 3:
                pass
            elif line.startswith("====="):
                eidx += 1
            elif line.startswith("eid"):
                fields = line.strip().split(',')
                gold = int(fields[1].split("=")[1])
                preds = list(map(int, fields[-1].strip().split()))
                all_preds_per_example.append(preds)
                all_golds.append(gold)
                for pid, pp in enumerate(preds):
                    all_preds_per_prompt[pid].append(pp)
            else:
                # read pred probs
                pass
    return num_prompts, num_examples, all_preds_per_example, all_preds_per_prompt, all_golds, prompts_accuracy, max_acc, median_acc, min_acc, ensemble_acc

test_file_names = ['anli_none_dev_r1_bigscience.T0', 'anli_none_dev_r2_bigscience.T0', 'anli_none_dev_r3_bigscience.T0', 
                  'hellaswag_none_validation_bigscience.T0', 'super_glue_cb_validation_bigscience.T0',
                  'super_glue_copa_validation_bigscience.T0', 'super_glue_rte_validation_bigscience.T0',
                  'super_glue_wic_validation_bigscience.T0', 'super_glue_wsc.fixed_validation_bigscience.T0', 'winogrande_winogrande_xl_validation_bigscience.T0']

t0_11b = {}
for fname in test_file_names:
    num_prompts, num_examples, all_preds_per_example, all_preds_per_prompt, all_golds, prompts_accuracy, max_acc, median_acc, min_acc, ensemble_acc = read_file(fname)
    t0_11b[fname] = {"nprompts": num_prompts, 'nexamples': num_examples, 'all_preds_per_example': all_preds_per_example, 'all_preds_per_prompt': all_preds_per_prompt, "all_golds": all_golds, 
                    "prompts_accuracy": prompts_accuracy, 'max_acc': max_acc, 'median_acc': median_acc, 'min_acc': min_acc, 'ensemble_acc': ensemble_acc}
    
t0_3b = {}
for fname in test_file_names:
    num_prompts, num_examples, all_preds_per_example, all_preds_per_prompt, all_golds, prompts_accuracy, max_acc, median_acc, min_acc, ensemble_acc = read_file(fname+"_3B")
    t0_3b[fname] = {"nprompts": num_prompts, 'nexamples': num_examples, 'all_preds_per_example': all_preds_per_example, 'all_preds_per_prompt': all_preds_per_prompt, "all_golds": all_golds, 
                    "prompts_accuracy": prompts_accuracy, 'max_acc': max_acc, 'median_acc': median_acc, 'min_acc': min_acc, 'ensemble_acc': ensemble_acc}
    

def cal_task_prompt_oracle_acc(task_dict, suffix=""):
    for fname in test_file_names:
        results = task_dict[fname]
        degenerate_prompts = set()
        for pid, preds in enumerate(task_dict[fname]['all_preds_per_prompt']):
            if len(set(preds)) == 1:
                degenerate_prompts.add(pid)
            else:
                count = Counter(preds)
                if max(np.array(list(count.values()))*1.0 / sum(count.values())) > 0.8:
                    degenerate_prompts.add(pid)
            
        all_preds_per_example = task_dict[fname]['all_preds_per_example']
        all_golds = task_dict[fname]['all_golds']
        print(degenerate_prompts)
        count = 0
        for pred_per_example, gold in zip(all_preds_per_example, all_golds):
            pred_per_example = [pred_per_example[i] for i in range(len(all_preds_per_example[0])) if i not in degenerate_prompts]
            count += 1 if gold in pred_per_example else 0
#             if pred_per_example.count(gold) >= 2:
#                 count += 1
        oracle_acc = count * 100. / len(all_golds)
        max_acc, median_acc, min_acc, ensemble_acc = task_dict[fname]['max_acc'], task_dict[fname]['median_acc'], task_dict[fname]['min_acc'], task_dict[fname]['ensemble_acc'], 
        print("{}{}: max_acc = {}, oracle_acc = {}, median_acc = {}, min_acc = {}, ensemble_acc = {}".format(fname, suffix, max_acc, oracle_acc, median_acc, min_acc, ensemble_acc))
        
cal_task_prompt_oracle_acc(t0_11b)
cal_task_prompt_oracle_acc(t0_3b, "_3B")

def cal_correlation():
    t11b_accs, t3b_accs = [], []
    for fname in test_file_names:
        t11b_accs.extend(t0_11b[fname]["prompts_accuracy"])
        t3b_accs.extend(t0_3b[fname]["prompts_accuracy"])
    corr = scipy.stats.pearsonr(t11b_accs, t3b_accs)
    print("correlation = {}".format(corr))

cal_correlation()

{10}
anli_none_dev_r1_bigscience.T0: max_acc = 46.5, oracle_acc = 80.2, median_acc = 44.7, min_acc = 33.3, ensemble_acc = 45.5
{10}
anli_none_dev_r2_bigscience.T0: max_acc = 40.8, oracle_acc = 78.9, median_acc = 39.7, min_acc = 33.5, ensemble_acc = 39.9
{10}
anli_none_dev_r3_bigscience.T0: max_acc = 44.25, oracle_acc = 79.58333333333333, median_acc = 42.67, min_acc = 33.92, ensemble_acc = 43.33
set()
hellaswag_none_validation_bigscience.T0: max_acc = 34.13, oracle_acc = 64.35968930491934, median_acc = 33.67, min_acc = 33.18, ensemble_acc = 34.8
{10}
super_glue_cb_validation_bigscience.T0: max_acc = 80.36, oracle_acc = 98.21428571428571, median_acc = 78.57, min_acc = 8.93, ensemble_acc = 78.57
set()
super_glue_copa_validation_bigscience.T0: max_acc = 96.0, oracle_acc = 99.0, median_acc = 93.0, min_acc = 82.0, ensemble_acc = 94.0
set()
super_glue_rte_validation_bigscience.T0: max_acc = 84.84, oracle_acc = 93.14079422382672, median_acc = 82.31, min_acc = 73.65, ensemble_acc = 84.48
{1, 3,

In [3]:
! export TRANSFORMERS_CACHE=../pretrain_models/huggingface
! export HF_DATASETS_CACHE=../pretrain_models/huggingface
! export HF_METRICS_CACHE=../pretrain_models/huggingface
! cache_dir=../pretrain_models/huggingface

# T0 train set

from os import read
from promptsource.templates import DatasetTemplates
import datasets
cache_dir='../pretrain_models/huggingface'

all_datasets = [('glue', 'mrpc', 'validation'), ('glue', 'qqp', 'validation'), ('paws', 'labeled_final', 'validation'), ('ai2_arc', 'ARC-Challenge', 'validation'), ('ai2_arc', 'ARC-Easy', 'validation'), ('kilt_tasks', 'hotpotqa', 'validation'), ('trivia_qa', 'unfiltered', 'validation'), ('web_questions', None, 'test'), ('wiki_qa', None, 'validation'), ('adversarial_qa', 'dbidaf', 'validation'), ('adversarial_qa', 'dbert', 'validation'), ('adversarial_qa', 'droberta', 'validation'), ('duorc', 'SelfRC', 'validation'), ('duorc', 'ParaphraseRC', 'validation'), ('ropes', None, 'validation'), ('squad_v2', None, 'validation'), ('super_glue', 'record', 'validation'), ('quoref', None, 'validation'), ('tydiqa', 'primary_task', 'validation'), ('cos_e', 'v1.11', 'validation'), ('cosmos_qa', None, 'validation'), ('dream', None, 'validation'), ('openbookqa', 'main', 'validation'), ('qasc', None, 'validation'), ('quail', None, 'validation'), ('quarel', None, 'validation'), ('quartz', None, 'validation'), ('race', 'high', 'validation'), ('race', 'middle', 'validation'), ('sciq', None, 'validation'), ('social_i_qa', None, 'validation'), ('super_glue', 'boolq', 'validation'), ('super_glue', 'multirc', 'validation'), ('wiki_hop', 'original', 'validation'), ('wiqa', None, 'validation'), ('piqa', None, 'validation'), ('amazon_polarity', None, 'test'), ('app_reviews', None, 'train'), ('imdb', None, 'test'), ('rotten_tomatoes', None, 'validation'), ('yelp_review_full', None, 'test'), ('common_gen', None, 'validation'), ('wiki_bio', None, 'val'), ('cnn_dailymail', '3.0.0', 'validation'), ('gigaword', None, 'validation'), ('multi_news', None, 'validation'), ('xsum', None, 'validation'), ('samsum', None, 'validation'), ('ag_news', None, 'test'), ('dbpedia_14', None, 'test'), ('trec', None, 'test')]

datasets_with_more_than_2_original_prompts = 0
datasets_with_only_nonoriginal_prompts = 0

def read_prompts(dataset, subset, split):
    global datasets_with_more_than_2_original_prompts
    global datasets_with_only_nonoriginal_prompts
    print()
    print("==="*100)
    print(dataset, subset, split)
    data = datasets.load_dataset(dataset, subset, cache_dir=cache_dir)
    print(data.keys())
    data = data[split]
    print(len(data))
    prompts = DatasetTemplates(dataset, subset)
    prompt_names = prompts.all_template_names
    original_prompts =[n for n in prompt_names if prompts[n].metadata.original_task]
    non_original_prompts = [n for n in prompt_names if not prompts[n].metadata.original_task]
    print("total prompts = {}, original prompts = {}".format(len(prompt_names), len(original_prompts)))

    # choices = prompts[prompt_names[0]].get_answer_choices_list(data[0])
    # print(data[0].keys())
    # if choices is not None:
    #     print("number of choices = {}, {}".format(len(choices), choices))
    # else:
    #     print(">>>>>>>>>>>>>>>>>>>>>>>>>>>> no choices >>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(data[0].keys())
    print(data[0])
    if len(original_prompts) > 0:
        print(prompts[original_prompts[0]].apply(data[0]))
        for pp in original_prompts:
            print('\noriginal =========== {}: {} ==========='.format(pp, prompts[pp].metadata.metrics))
            print(prompts[pp].jinja)
            choices = prompts[pp].get_answer_choices_list(data[0])
            if choices is not None:
                print("++++++")
                print(choices)
                print("++++++")
            print(prompts[pp].apply(data[0]))
            print(prompts[pp].apply(data[4]))
    if len(non_original_prompts) > 0:
        for pp in non_original_prompts:
            print('\nnon original =========== {}.{} ==========='.format(pp, prompts[pp].metadata.metrics))
            print(prompts[pp].jinja)
            choices = prompts[pp].get_answer_choices_list(data[0])
            if choices is not None:
                print("++++++")
                print(len(choices), choices)
                print("++++++")
            print(prompts[pp].apply(data[0]))
            print(prompts[pp].apply(data[4]))
    
    if len(original_prompts) >= 3:
        datasets_with_more_than_2_original_prompts += 1
    if len(original_prompts) == 0:
        datasets_with_only_nonoriginal_prompts += 1

read_prompts("app_reviews", None, "train")
# a = 50
# b = 51
# for dataset, subset, split in all_datasets[a:b]:
#     print(dataset, subset, split)
#     read_prompts(dataset, subset, split)


print("datasets with more than 2 original prompts: {}".format(datasets_with_more_than_2_original_prompts))
print("datasets with only non-original prompts: {}".format(datasets_with_only_nonoriginal_prompts))


app_reviews None train


Using custom data configuration default
Reusing dataset app_reviews (../pretrain_models/huggingface/app_reviews/default/0.0.0/20335b51b604b9bc04b7be253cd8445caa9ba93f15f39a4b0492b9e9102853de)
100%|██████████| 1/1 [00:00<00:00, 439.93it/s]

dict_keys(['train'])
288065
total prompts = 4, original prompts = 0
dict_keys(['package_name', 'review', 'date', 'star'])
{'package_name': 'com.mantz_it.rfanalyzer', 'review': "Great app! The new version now works on my Bravia Android TV which is great as it's right by my rooftop aerial cable. The scan feature would be useful...any ETA on when this will be available? Also the option to import a list of bookmarks e.g. from a simple properties file would be useful.", 'date': 'October 12 2016', 'star': 4}

non original =========== categorize_rating_using_review.['Accuracy', 'Spearman Correlation'] ===========
Given this review: "{{review}}"
Would you recommend this app to a friend? {{answer_choices[0]}}, {{answer_choices[1]}}, {{answer_choices[2]}}, {{answer_choices[3]}}, or {{answer_choices[4]}}?
|||
{{answer_choices[star-1]}}
++++++
5 ['Not at all', 'No', 'Maybe', 'Yes', 'Definitely']
++++++
['Given this review: "Great app! The new version now works on my Bravia Android TV which is grea

In [7]:
! export TRANSFORMERS_CACHE=../pretrain_models/huggingface
! export HF_DATASETS_CACHE=../pretrain_models/huggingface
! export HF_METRICS_CACHE=../pretrain_models/huggingface
! cache_dir=../pretrain_models/huggingface
       
# read_prompts("hotpot_qa", "fullwiki", "validation")

data = datasets.load_dataset('bigscience/P3', "trivia_qa_unfiltered_first_person_context", cache_dir=cache_dir)["validation"]
print(data[0])

2022-09-23 11:44:03.906098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 11:44:04.016803: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-23 11:44:04.022006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /private/home/chuntinz/work/tools/lib/::/public/slurm/20.11.8/lib:/public/slurm/20.11.8/lib
2022

KeyboardInterrupt: 

In [ ]:
configs = ['adversarial_qa_dbert_answer_the_following_q', 'adversarial_qa_dbert_based_on', 'adversarial_qa_dbert_generate_question', 'adversarial_qa_dbert_question_context_answer', 'adversarial_qa_dbert_tell_what_it_is', 'adversarial_qa_dbidaf_answer_the_following_q', 'adversarial_qa_dbidaf_based_on', 'adversarial_qa_dbidaf_generate_question', 'adversarial_qa_dbidaf_question_context_answer', 'adversarial_qa_dbidaf_tell_what_it_is', 'adversarial_qa_droberta_answer_the_following_q', 'adversarial_qa_droberta_based_on', 'adversarial_qa_droberta_generate_question', 'adversarial_qa_droberta_question_context_answer', 'adversarial_qa_droberta_tell_what_it_is', 'ag_news_classify', 'ag_news_classify_question_first', 'ag_news_classify_with_choices', 'ag_news_classify_with_choices_question_first', 'ag_news_recommend', 'ag_news_which_section', 'ag_news_which_section_choices', 'ai2_arc_ARC_Challenge_heres_a_problem', 'ai2_arc_ARC_Challenge_i_am_hesitating', 'ai2_arc_ARC_Challenge_multiple_choice', 'ai2_arc_ARC_Challenge_pick_false_options', 'ai2_arc_ARC_Challenge_pick_the_most_correct_option', 'ai2_arc_ARC_Challenge_qa_options', 'ai2_arc_ARC_Easy_heres_a_problem', 'ai2_arc_ARC_Easy_i_am_hesitating', 'ai2_arc_ARC_Easy_multiple_choice', 'ai2_arc_ARC_Easy_pick_false_options', 'ai2_arc_ARC_Easy_pick_the_most_correct_option', 'ai2_arc_ARC_Easy_qa_options', 'amazon_polarity_Is_this_product_review_positive', 'amazon_polarity_Is_this_review', 'amazon_polarity_Is_this_review_negative', 'amazon_polarity_User_recommend_this_product', 'amazon_polarity_convey_negative_or_positive_sentiment', 'amazon_polarity_flattering_or_not', 'amazon_polarity_negative_or_positive_tone', 'amazon_polarity_user_satisfied', 'amazon_polarity_would_you_buy', 'anli_GPT_3_style_r1', 'anli_GPT_3_style_r1_score_eval', 'anli_GPT_3_style_r2', 'anli_GPT_3_style_r2_score_eval', 'anli_GPT_3_style_r3', 'anli_GPT_3_style_r3_score_eval', 'anli_MNLI_crowdsource_r1', 'anli_MNLI_crowdsource_r1_score_eval', 'anli_MNLI_crowdsource_r2', 'anli_MNLI_crowdsource_r2_score_eval', 'anli_MNLI_crowdsource_r3', 'anli_MNLI_crowdsource_r3_score_eval', 'anli_always_sometimes_never_r1', 'anli_always_sometimes_never_r1_score_eval', 'anli_always_sometimes_never_r2', 'anli_always_sometimes_never_r2_score_eval', 'anli_always_sometimes_never_r3', 'anli_always_sometimes_never_r3_score_eval', 'anli_based_on_the_previous_passage_r1', 'anli_based_on_the_previous_passage_r1_score_eval', 'anli_based_on_the_previous_passage_r2', 'anli_based_on_the_previous_passage_r2_score_eval', 'anli_based_on_the_previous_passage_r3', 'anli_based_on_the_previous_passage_r3_score_eval', 'anli_can_we_infer_r1', 'anli_can_we_infer_r1_score_eval', 'anli_can_we_infer_r2', 'anli_can_we_infer_r2_score_eval', 'anli_can_we_infer_r3', 'anli_can_we_infer_r3_score_eval', 'anli_claim_true_false_inconclusive_r1', 'anli_claim_true_false_inconclusive_r1_score_eval', 'anli_claim_true_false_inconclusive_r2', 'anli_claim_true_false_inconclusive_r2_score_eval', 'anli_claim_true_false_inconclusive_r3', 'anli_claim_true_false_inconclusive_r3_score_eval', 'anli_consider_always_sometimes_never_r1', 'anli_consider_always_sometimes_never_r1_score_eval', 'anli_consider_always_sometimes_never_r2', 'anli_consider_always_sometimes_never_r2_score_eval', 'anli_consider_always_sometimes_never_r3', 'anli_consider_always_sometimes_never_r3_score_eval', 'anli_does_it_follow_that_r1', 'anli_does_it_follow_that_r1_score_eval', 'anli_does_it_follow_that_r2', 'anli_does_it_follow_that_r2_score_eval', 'anli_does_it_follow_that_r3', 'anli_does_it_follow_that_r3_score_eval', 'anli_does_this_imply_r1', 'anli_does_this_imply_r1_score_eval', 'anli_does_this_imply_r2', 'anli_does_this_imply_r2_score_eval', 'anli_does_this_imply_r3', 'anli_does_this_imply_r3_score_eval', 'anli_guaranteed_possible_impossible_r1', 'anli_guaranteed_possible_impossible_r1_score_eval', 'anli_guaranteed_possible_impossible_r2', 'anli_guaranteed_possible_impossible_r2_score_eval', 'anli_guaranteed_possible_impossible_r3', 'anli_guaranteed_possible_impossible_r3_score_eval', 'anli_guaranteed_true_r1', 'anli_guaranteed_true_r1_score_eval', 'anli_guaranteed_true_r2', 'anli_guaranteed_true_r2_score_eval', 'anli_guaranteed_true_r3', 'anli_guaranteed_true_r3_score_eval', 'anli_justified_in_saying_r1', 'anli_justified_in_saying_r1_score_eval', 'anli_justified_in_saying_r2', 'anli_justified_in_saying_r2_score_eval', 'anli_justified_in_saying_r3', 'anli_justified_in_saying_r3_score_eval', 'anli_must_be_true_r1', 'anli_must_be_true_r1_score_eval', 'anli_must_be_true_r2', 'anli_must_be_true_r2_score_eval', 'anli_must_be_true_r3', 'anli_must_be_true_r3_score_eval', 'anli_should_assume_r1', 'anli_should_assume_r1_score_eval', 'anli_should_assume_r2', 'anli_should_assume_r2_score_eval', 'anli_should_assume_r3', 'anli_should_assume_r3_score_eval', 'anli_take_the_following_as_truth_r1', 'anli_take_the_following_as_truth_r1_score_eval', 'anli_take_the_following_as_truth_r2', 'anli_take_the_following_as_truth_r2_score_eval', 'anli_take_the_following_as_truth_r3', 'anli_take_the_following_as_truth_r3_score_eval', 'app_reviews_categorize_rating_using_review', 'app_reviews_convert_to_rating', 'app_reviews_convert_to_star_rating', 'app_reviews_generate_review', 'cnn_dailymail_3.0.0_2_or_3_sentences', 'cnn_dailymail_3.0.0_generate_story', 'cnn_dailymail_3.0.0_news_card_view', 'cnn_dailymail_3.0.0_news_stock', 'cnn_dailymail_3.0.0_news_summary', 'cnn_dailymail_3.0.0_spice_up_story', 'cnn_dailymail_3.0.0_sum_in_brief', 'cnn_dailymail_3.0.0_tldr_summary', 'cnn_dailymail_3.0.0_write_an_outline', 'common_gen_Example_prompt', 'common_gen_Given_concepts_type_1', 'common_gen_Given_concepts_type_2', 'common_gen_Put_together', 'common_gen_choice_in_concept_centric_sentence_generation', 'common_gen_random_task_template_prompt', 'common_gen_sentence_to_concepts', 'common_gen_topic_to_sentence', 'common_gen_topics_from_the_sentence', 'cos_e_v1.11_aligned_with_common_sense', 'cos_e_v1.11_description_question_option_id', 'cos_e_v1.11_description_question_option_text', 'cos_e_v1.11_explain_why_human', 'cos_e_v1.11_generate_explanation_given_text', 'cos_e_v1.11_i_think', 'cos_e_v1.11_question_description_option_id', 'cos_e_v1.11_question_description_option_text', 'cos_e_v1.11_question_option_description_id', 'cos_e_v1.11_question_option_description_text', 'cos_e_v1.11_rationale', 'cosmos_qa_context_answer_to_question', 'cosmos_qa_context_description_question_answer_id', 'cosmos_qa_context_description_question_answer_text', 'cosmos_qa_context_description_question_text', 'cosmos_qa_context_question_description_answer_id', 'cosmos_qa_context_question_description_answer_text', 'cosmos_qa_context_question_description_text', 'cosmos_qa_description_context_question_answer_id', 'cosmos_qa_description_context_question_answer_text', 'cosmos_qa_description_context_question_text', 'cosmos_qa_no_prompt_id', 'cosmos_qa_no_prompt_text', 'cosmos_qa_only_question_answer', 'dbpedia_14_given_a_choice_of_categories_', 'dbpedia_14_given_a_list_of_category_what_does_the_title_belong_to', 'dbpedia_14_given_list_what_category_does_the_paragraph_belong_to', 'dbpedia_14_pick_one_category_for_the_following_text', 'dream_answer_to_dialogue', 'dream_baseline', 'dream_generate_first_utterance', 'dream_generate_last_utterance', 'dream_read_the_following_conversation_and_answer_the_question', 'duorc_ParaphraseRC_answer_question', 'duorc_ParaphraseRC_build_story_around_qa', 'duorc_ParaphraseRC_decide_worth_it', 'duorc_ParaphraseRC_extract_answer', 'duorc_ParaphraseRC_generate_question', 'duorc_ParaphraseRC_generate_question_by_answer', 'duorc_ParaphraseRC_movie_director', 'duorc_ParaphraseRC_question_answering', 'duorc_ParaphraseRC_title_generation', 'duorc_SelfRC_answer_question', 'duorc_SelfRC_build_story_around_qa', 'duorc_SelfRC_decide_worth_it', 'duorc_SelfRC_extract_answer', 'duorc_SelfRC_generate_question', 'duorc_SelfRC_generate_question_by_answer', 'duorc_SelfRC_movie_director', 'duorc_SelfRC_question_answering', 'duorc_SelfRC_title_generation', 'gigaword_TLDR', 'gigaword_first_sentence_title', 'gigaword_generate_summary_for_this', 'gigaword_in_a_nutshell', 'gigaword_make_a_title', 'gigaword_reverse_writing', 'gigaword_write_a_title_for_this_sentence', 'gigaword_write_an_article', 'gigaword_write_its_sentence', 'glue_mrpc_equivalent', 'glue_mrpc_generate_paraphrase', 'glue_mrpc_generate_sentence', 'glue_mrpc_paraphrase', 'glue_mrpc_replace', 'glue_mrpc_same_thing', 'glue_mrpc_want_to_know', 'glue_qqp_answer', 'glue_qqp_duplicate', 'glue_qqp_duplicate_or_not', 'glue_qqp_meaning', 'glue_qqp_quora', 'glue_qqp_same_thing', 'hellaswag_Appropriate_continuation_Yes_or_No', 'hellaswag_Open_ended_completion', 'hellaswag_Open_ended_start', 'hellaswag_Predict_ending_with_hint', 'hellaswag_Predict_ending_with_hint_score_eval', 'hellaswag_Randomized_prompts_template', 'hellaswag_Randomized_prompts_template_score_eval', 'hellaswag_Reversed_appropriate_continuation_Yes_or_No', 'hellaswag_Topic_of_the_context', 'hellaswag_Topic_without_the_ending_answer', 'hellaswag_complete_first_then', 'hellaswag_complete_first_then_score_eval', 'hellaswag_how_ends', 'hellaswag_if_begins_how_continues', 'hellaswag_if_begins_how_continues_score_eval', 'imdb_Movie_Expressed_Sentiment', 'imdb_Movie_Expressed_Sentiment_2', 'imdb_Negation_template_for_positive_and_negative', 'imdb_Reviewer_Enjoyment', 'imdb_Reviewer_Enjoyment_Yes_No', 'imdb_Reviewer_Expressed_Sentiment', 'imdb_Reviewer_Opinion_bad_good_choices', 'imdb_Reviewer_Sentiment_Feeling', 'imdb_Sentiment_with_choices_', 'imdb_Text_Expressed_Sentiment', 'imdb_Writer_Expressed_Sentiment', 'kilt_tasks_hotpotqa_combining_facts', 'kilt_tasks_hotpotqa_complex_question', 'kilt_tasks_hotpotqa_final_exam', 'kilt_tasks_hotpotqa_formulate', 'kilt_tasks_hotpotqa_straighforward_qa', 'multi_news_distill', 'multi_news_expand_reverse_task_', 'multi_news_summarize', 'multi_news_summary_scenario', 'multi_news_synthesize', 'multi_news_what_are_the_key_points', 'openbookqa_main_choices', 'openbookqa_main_choose_an_answer_with_options', 'openbookqa_main_only_options', 'openbookqa_main_pick_answer_with_options', 'openbookqa_main_pick_using_id', 'openbookqa_main_which_correct', 'openbookqa_main_which_correct_inverse', 'paws_labeled_final_Concatenation', 'paws_labeled_final_Concatenation_no_label', 'paws_labeled_final_Meaning', 'paws_labeled_final_Meaning_no_label', 'paws_labeled_final_PAWS_ANLI_GPT3', 'paws_labeled_final_PAWS_ANLI_GPT3_no_label', 'paws_labeled_final_Rewrite', 'paws_labeled_final_Rewrite_no_label', 'paws_labeled_final_context_question', 'paws_labeled_final_context_question_no_label', 'paws_labeled_final_paraphrase_task', 'paws_labeled_final_task_description_no_label', 'piqa_Correct_the_solution', 'piqa_Correct_the_solution_if_false_from_sol_1', 'piqa_Correct_the_solution_if_false_from_sol_2', 'piqa_Does_this_solution_make_sense_sol1', 'piqa_Does_this_solution_make_sense_sol2', 'piqa_choose_the_most_appropriate_solution', 'piqa_finish_sentence_with_correct_choice', 'piqa_no_prompt_needed', 'piqa_pick_correct_choice_index', 'piqa_pick_correct_choice_with_choice_given_before_goal', 'piqa_what_is_the_correct_ending', 'qasc_is_correct_1', 'qasc_is_correct_2', 'qasc_qa_with_combined_facts_1', 'qasc_qa_with_separated_facts_1', 'qasc_qa_with_separated_facts_2', 'qasc_qa_with_separated_facts_3', 'qasc_qa_with_separated_facts_4', 'qasc_qa_with_separated_facts_5', 'quail_context_description_question_answer_id', 'quail_context_description_question_answer_text', 'quail_context_description_question_text', 'quail_context_question_answer_description_id', 'quail_context_question_answer_description_text', 'quail_context_question_description_answer_id', 'quail_context_question_description_answer_text', 'quail_context_question_description_text', 'quail_description_context_question_answer_id', 'quail_description_context_question_answer_text', 'quail_description_context_question_text', 'quail_no_prompt_id', 'quail_no_prompt_text', 'quarel_choose_between', 'quarel_do_not_use', 'quarel_heres_a_story', 'quarel_logic_test', 'quarel_testing_students', 'quartz_answer_question_based_on', 'quartz_answer_question_below', 'quartz_given_the_fact_answer_the_q', 'quartz_having_read_above_passage', 'quartz_paragraph_question_plain_concat', 'quartz_read_passage_below_choose', 'quartz_use_info_from_paragraph_question', 'quartz_use_info_from_question_paragraph', 'quoref_Answer_Friend_Question', 'quoref_Answer_Question_Given_Context', 'quoref_Answer_Test', 'quoref_Context_Contains_Answer', 'quoref_Find_Answer', 'quoref_Found_Context_Online', 'quoref_Given_Context_Answer_Question', 'quoref_Guess_Answer', 'quoref_Guess_Title_For_Context', 'quoref_Read_And_Extract_', 'quoref_What_Is_The_Answer', 'race_high_Is_this_the_right_answer', 'race_high_Read_the_article_and_answer_the_question_no_option_', 'race_high_Select_the_best_answer', 'race_high_Select_the_best_answer_generate_span_', 'race_high_Select_the_best_answer_no_instructions_', 'race_high_Taking_a_test', 'race_high_Write_a_multi_choice_question_for_the_following_article', 'race_high_Write_a_multi_choice_question_options_given_', 'race_middle_Is_this_the_right_answer', 'race_middle_Read_the_article_and_answer_the_question_no_option_', 'race_middle_Select_the_best_answer', 'race_middle_Select_the_best_answer_generate_span_', 'race_middle_Select_the_best_answer_no_instructions_', 'race_middle_Taking_a_test', 'race_middle_Write_a_multi_choice_question_for_the_following_article', 'race_middle_Write_a_multi_choice_question_options_given_', 'ropes_background_new_situation_answer', 'ropes_background_situation_middle', 'ropes_given_background_situation', 'ropes_new_situation_background_answer', 'ropes_plain_background_situation', 'ropes_plain_bottom_hint', 'ropes_plain_no_background', 'ropes_prompt_beginning', 'ropes_prompt_bottom_hint_beginning', 'ropes_prompt_bottom_no_hint', 'ropes_prompt_mix', 'ropes_read_background_situation', 'rotten_tomatoes_Movie_Expressed_Sentiment', 'rotten_tomatoes_Movie_Expressed_Sentiment_2', 'rotten_tomatoes_Reviewer_Enjoyment', 'rotten_tomatoes_Reviewer_Enjoyment_Yes_No', 'rotten_tomatoes_Reviewer_Expressed_Sentiment', 'rotten_tomatoes_Reviewer_Opinion_bad_good_choices', 'rotten_tomatoes_Reviewer_Sentiment_Feeling', 'rotten_tomatoes_Sentiment_with_choices_', 'rotten_tomatoes_Text_Expressed_Sentiment', 'rotten_tomatoes_Writer_Expressed_Sentiment', 'samsum_Generate_a_summary_for_this_dialogue', 'samsum_Given_the_above_dialogue_write_a_summary', 'samsum_Sum_up_the_following_dialogue', 'samsum_Summarize_', 'samsum_Summarize_this_dialogue_', 'samsum_To_sum_up_this_dialog', 'samsum_Write_a_dialogue_that_match_this_summary', 'sciq_Direct_Question', 'sciq_Direct_Question_Closed_Book_', 'sciq_Multiple_Choice', 'sciq_Multiple_Choice_Closed_Book_', 'sciq_Multiple_Choice_Question_First', 'social_i_qa_Check_if_a_random_answer_is_valid_or_not', 'social_i_qa_Generate_answer', 'social_i_qa_Generate_the_question_from_the_answer', 'social_i_qa_I_was_wondering', 'social_i_qa_Show_choices_and_generate_answer', 'social_i_qa_Show_choices_and_generate_index', 'squad_v2_Jeopardy_with_Context', 'squad_v2_Jeopardy_without_Context', 'squad_v2_Questions_with_Context', 'squad_v2_Questions_with_Context_Without_Prompt_Keywords', 'squad_v2_Questions_with_Context_Without_Prompt_Keywords_unanswerable', 'squad_v2_Questions_with_Context_unanswerable', 'squad_v2_Topic_Prediction_Context', 'squad_v2_Topic_Prediction_Context_with_randomized_prompt_options', 'squad_v2_Topic_Prediction_Context_with_randomized_prompt_options_placed_in_the_end', 'squad_v2_Topic_Prediction_Question_and_Answer_Pair', 'squad_v2_Trivia', 'squad_v2_Unanwerable_question', 'super_glue_boolq_GPT_3_Style', 'super_glue_boolq_I_wonder_', 'super_glue_boolq_after_reading', 'super_glue_boolq_based_on_the_following_passage', 'super_glue_boolq_based_on_the_previous_passage', 'super_glue_boolq_could_you_tell_me_', 'super_glue_boolq_exam', 'super_glue_boolq_exercise', 'super_glue_boolq_valid_binary', 'super_glue_boolq_yes_no_question', 'super_glue_cb_GPT_3_style', 'super_glue_cb_GPT_3_style_score_eval', 'super_glue_cb_MNLI_crowdsource', 'super_glue_cb_MNLI_crowdsource_score_eval', 'super_glue_cb_always_sometimes_never', 'super_glue_cb_always_sometimes_never_score_eval', 'super_glue_cb_based_on_the_previous_passage', 'super_glue_cb_based_on_the_previous_passage_score_eval', 'super_glue_cb_can_we_infer', 'super_glue_cb_can_we_infer_score_eval', 'super_glue_cb_claim_true_false_inconclusive', 'super_glue_cb_claim_true_false_inconclusive_score_eval', 'super_glue_cb_consider_always_sometimes_never', 'super_glue_cb_consider_always_sometimes_never_score_eval', 'super_glue_cb_does_it_follow_that', 'super_glue_cb_does_it_follow_that_score_eval', 'super_glue_cb_does_this_imply', 'super_glue_cb_does_this_imply_score_eval', 'super_glue_cb_guaranteed_possible_impossible', 'super_glue_cb_guaranteed_possible_impossible_score_eval', 'super_glue_cb_guaranteed_true', 'super_glue_cb_guaranteed_true_score_eval', 'super_glue_cb_justified_in_saying', 'super_glue_cb_justified_in_saying_score_eval', 'super_glue_cb_must_be_true', 'super_glue_cb_must_be_true_score_eval', 'super_glue_cb_should_assume', 'super_glue_cb_should_assume_score_eval', 'super_glue_cb_take_the_following_as_truth', 'super_glue_cb_take_the_following_as_truth_score_eval', 'super_glue_copa_C1_or_C2_premise_so_because_', 'super_glue_copa_C1_or_C2_premise_so_because__score_eval', 'super_glue_copa__As_a_result_C1_or_C2_', 'super_glue_copa__As_a_result_C1_or_C2__score_eval', 'super_glue_copa__What_could_happen_next_C1_or_C2_', 'super_glue_copa__What_could_happen_next_C1_or_C2__score_eval', 'super_glue_copa__which_may_be_caused_by', 'super_glue_copa__which_may_be_caused_by_score_eval', 'super_glue_copa__why_C1_or_C2', 'super_glue_copa__why_C1_or_C2_score_eval', 'super_glue_copa_best_option', 'super_glue_copa_best_option_score_eval', 'super_glue_copa_cause_effect', 'super_glue_copa_cause_effect_score_eval', 'super_glue_copa_choose', 'super_glue_copa_choose_score_eval', 'super_glue_copa_exercise', 'super_glue_copa_exercise_score_eval', 'super_glue_copa_i_am_hesitating', 'super_glue_copa_i_am_hesitating_score_eval', 'super_glue_copa_more_likely', 'super_glue_copa_more_likely_score_eval', 'super_glue_copa_plausible_alternatives', 'super_glue_copa_plausible_alternatives_score_eval', 'super_glue_multirc_I_was_going_to_say_', 'super_glue_multirc_Would_it_be_good_to_answer_', 'super_glue_multirc_confirm', 'super_glue_multirc_correct', 'super_glue_multirc_decide_valid', 'super_glue_multirc_found_this_answer', 'super_glue_multirc_grading', 'super_glue_multirc_is_a_correct_answer_', 'super_glue_multirc_is_the_correct_answer_', 'super_glue_multirc_paragraph_question_is_it_', 'super_glue_record_Add_sentence_after_after_continuation_choices_', 'super_glue_record_Add_sentence_after_continuation_choices_', 'super_glue_record_Can_you_figure_out_', 'super_glue_record_GPT_3_style_continuation_choices_', 'super_glue_record_GPT_3_style_summary_only_continuation_choices_', 'super_glue_record_GPT_3_style_with_labels_continuation_choices_', 'super_glue_record_GPT_3_style_with_labels_without_hyphens_continuation_choices_', 'super_glue_record_GPT_3_style_without_hyphens_continuation_choices_', 'super_glue_record_In_the_question_above_the_placeholder_stands_for', 'super_glue_record_New_highlight_continuation_choices_', 'super_glue_record_News_article_continuation_choices_', 'super_glue_record_Summary_first_continuation_choices_', 'super_glue_record_What_could_the_placeholder_be_', 'super_glue_record_Which_one_is_the_placeholder_', 'super_glue_record_choose_between', 'super_glue_record_corrupted', 'super_glue_record_exercise', 'super_glue_record_pick_one_option', 'super_glue_record_the_placeholder_refers_to_', 'super_glue_record_trying_to_decide', 'super_glue_rte_GPT_3_style', 'super_glue_rte_GPT_3_style_score_eval', 'super_glue_rte_MNLI_crowdsource', 'super_glue_rte_MNLI_crowdsource_score_eval', 'super_glue_rte_based_on_the_previous_passage', 'super_glue_rte_based_on_the_previous_passage_score_eval', 'super_glue_rte_can_we_infer', 'super_glue_rte_can_we_infer_score_eval', 'super_glue_rte_does_it_follow_that', 'super_glue_rte_does_it_follow_that_score_eval', 'super_glue_rte_does_this_imply', 'super_glue_rte_does_this_imply_score_eval', 'super_glue_rte_guaranteed_true', 'super_glue_rte_guaranteed_true_score_eval', 'super_glue_rte_justified_in_saying', 'super_glue_rte_justified_in_saying_score_eval', 'super_glue_rte_must_be_true', 'super_glue_rte_must_be_true_score_eval', 'super_glue_rte_should_assume', 'super_glue_rte_should_assume_score_eval', 'super_glue_wic_GPT_3_prompt', 'super_glue_wic_GPT_3_prompt_score_eval', 'super_glue_wic_GPT_3_prompt_with_label', 'super_glue_wic_GPT_3_prompt_with_label_score_eval', 'super_glue_wic_affirmation_true_or_false', 'super_glue_wic_affirmation_true_or_false_score_eval', 'super_glue_wic_grammar_homework', 'super_glue_wic_grammar_homework_score_eval', 'super_glue_wic_polysemous', 'super_glue_wic_polysemous_score_eval', 'super_glue_wic_question_context', 'super_glue_wic_question_context_meaning', 'super_glue_wic_question_context_meaning_score_eval', 'super_glue_wic_question_context_meaning_with_label', 'super_glue_wic_question_context_meaning_with_label_score_eval', 'super_glue_wic_question_context_score_eval', 'super_glue_wic_same_sense', 'super_glue_wic_same_sense_score_eval', 'super_glue_wic_similar_sense', 'super_glue_wic_similar_sense_score_eval', 'super_glue_wsc.fixed_GPT_3_Style', 'super_glue_wsc.fixed_GPT_3_Style_score_eval', 'super_glue_wsc.fixed_I_think_they_mean', 'super_glue_wsc.fixed_I_think_they_mean_score_eval', 'super_glue_wsc.fixed_Who_or_what_is_are', 'super_glue_wsc.fixed_Who_or_what_is_are_score_eval', 'super_glue_wsc.fixed_by_p_they_mean', 'super_glue_wsc.fixed_by_p_they_mean_score_eval', 'super_glue_wsc.fixed_does_p_stand_for', 'super_glue_wsc.fixed_does_p_stand_for_score_eval', 'super_glue_wsc.fixed_does_the_pronoun_refer_to', 'super_glue_wsc.fixed_does_the_pronoun_refer_to_score_eval', 'super_glue_wsc.fixed_in_other_words', 'super_glue_wsc.fixed_in_other_words_score_eval', 'super_glue_wsc.fixed_p_is_are_r', 'super_glue_wsc.fixed_p_is_are_r_score_eval', 'super_glue_wsc.fixed_replaced_with', 'super_glue_wsc.fixed_replaced_with_score_eval', 'super_glue_wsc.fixed_the_pronoun_refers_to', 'super_glue_wsc.fixed_the_pronoun_refers_to_score_eval', 'trec_fine_grained_ABBR', 'trec_fine_grained_ABBR_context_first', 'trec_fine_grained_DESC', 'trec_fine_grained_DESC_context_first', 'trec_fine_grained_ENTY', 'trec_fine_grained_HUM', 'trec_fine_grained_HUM_context_first', 'trec_fine_grained_LOC', 'trec_fine_grained_LOC_context_first', 'trec_fine_grained_NUM', 'trec_fine_grained_NUM_context_first', 'trec_fine_grained_open', 'trec_fine_grained_open_context_first', 'trec_pick_the_best_descriptor', 'trec_trec1', 'trec_trec2', 'trec_what_category_best_describe', 'trec_which_category_best_describes', 'trivia_qa_unfiltered_first_person_context', 'trivia_qa_unfiltered_formal_description', 'trivia_qa_unfiltered_guess_question', 'trivia_qa_unfiltered_question_answer', 'trivia_qa_unfiltered_question_with_instruction', 'web_questions_get_the_answer', 'web_questions_potential_correct_answer', 'web_questions_question_answer', 'web_questions_short_general_knowledge_q', 'web_questions_whats_the_answer', 'wiki_bio_comprehension', 'wiki_bio_guess_person', 'wiki_bio_key_content', 'wiki_bio_what_content', 'wiki_bio_who', 'wiki_hop_original_choose_best_object_affirmative_1', 'wiki_hop_original_choose_best_object_affirmative_2', 'wiki_hop_original_choose_best_object_affirmative_3', 'wiki_hop_original_choose_best_object_interrogative_1', 'wiki_hop_original_choose_best_object_interrogative_2', 'wiki_hop_original_explain_relation', 'wiki_hop_original_generate_object', 'wiki_hop_original_generate_subject', 'wiki_hop_original_generate_subject_and_object', 'wiki_qa_Decide_good_answer', 'wiki_qa_Direct_Answer_to_Question', 'wiki_qa_Generate_Question_from_Topic', 'wiki_qa_Is_This_True_', 'wiki_qa_Jeopardy_style', 'wiki_qa_Topic_Prediction_Answer_Only', 'wiki_qa_Topic_Prediction_Question_Only', 'wiki_qa_Topic_Prediction_Question_and_Answer_Pair', 'wiki_qa_automatic_system', 'wiki_qa_exercise', 'wiki_qa_found_on_google', 'winogrande_winogrande_debiased_Replace', 'winogrande_winogrande_debiased_Replace_score_eval', 'winogrande_winogrande_debiased_does_underscore_refer_to', 'winogrande_winogrande_debiased_does_underscore_refer_to_score_eval', 'winogrande_winogrande_debiased_fill_in_the_blank', 'winogrande_winogrande_debiased_fill_in_the_blank_score_eval', 'winogrande_winogrande_debiased_stand_for', 'winogrande_winogrande_debiased_stand_for_score_eval', 'winogrande_winogrande_debiased_underscore_refer_to', 'winogrande_winogrande_debiased_underscore_refer_to_score_eval', 'winogrande_winogrande_xl_Replace', 'winogrande_winogrande_xl_Replace_score_eval', 'winogrande_winogrande_xl_does_underscore_refer_to', 'winogrande_winogrande_xl_does_underscore_refer_to_score_eval', 'winogrande_winogrande_xl_fill_in_the_blank', 'winogrande_winogrande_xl_fill_in_the_blank_score_eval', 'winogrande_winogrande_xl_stand_for', 'winogrande_winogrande_xl_stand_for_score_eval', 'winogrande_winogrande_xl_underscore_refer_to', 'winogrande_winogrande_xl_underscore_refer_to_score_eval', 'wiqa_does_the_supposed_perturbation_have_an_effect', 'wiqa_effect_with_label_answer', 'wiqa_effect_with_string_answer', 'wiqa_what_is_the_final_step_of_the_following_process', 'wiqa_what_is_the_missing_first_step', 'wiqa_what_might_be_the_first_step_of_the_process', 'wiqa_what_might_be_the_last_step_of_the_process', 'wiqa_which_of_the_following_is_the_supposed_perturbation', 'xsum_DOC_boils_down_to_simple_idea_that', 'xsum_DOC_given_above_write_one_sentence', 'xsum_DOC_how_would_you_rephrase_few_words', 'xsum_DOC_tldr', 'xsum_DOC_write_summary_of_above', 'xsum_article_DOC_summary', 'xsum_college_roommate_asked_DOC_so_I_recap', 'xsum_read_below_DOC_write_abstract', 'xsum_summarize_DOC', 'xsum_summarize_this_DOC_summary', 'yelp_review_full_based_on_that', 'yelp_review_full_format_rating', 'yelp_review_full_format_score', 'yelp_review_full_format_star', 'yelp_review_full_on_a_scale', 'yelp_review_full_so_i_would', 'yelp_review_full_this_place']

all_datasets = [('glue', 'mrpc', 'validation'), ('glue', 'qqp', 'validation'), ('paws', 'labeled_final', 'validation'), ('ai2_arc', 'ARC-Challenge', 'validation'), ('ai2_arc', 'ARC-Easy', 'validation'), ('kilt_tasks', 'hotpotqa', 'validation'), ('trivia_qa', 'unfiltered', 'validation'), ('web_questions', None, 'test'), ('wiki_qa', None, 'validation'), ('adversarial_qa', 'dbidaf', 'validation'), ('adversarial_qa', 'dbert', 'validation'), ('adversarial_qa', 'droberta', 'validation'), ('duorc', 'SelfRC', 'validation'), ('duorc', 'ParaphraseRC', 'validation'), ('ropes', None, 'validation'), ('squad_v2', None, 'validation'), ('super_glue', 'record', 'validation'), ('quoref', None, 'validation'), ('tydiqa', 'primary_task', 'validation'), ('cos_e', 'v1.11', 'validation'), ('cosmos_qa', None, 'validation'), ('dream', None, 'validation'), ('openbookqa', 'main', 'validation'), ('qasc', None, 'validation'), ('quail', None, 'validation'), ('quarel', None, 'validation'), ('quartz', None, 'validation'), ('race', 'high', 'validation'), ('race', 'middle', 'validation'), ('sciq', None, 'validation'), ('social_i_qa', None, 'validation'), ('super_glue', 'boolq', 'validation'), ('super_glue', 'multirc', 'validation'), ('wiki_hop', 'original', 'validation'), ('wiqa', None, 'validation'), ('piqa', None, 'validation'), ('amazon_polarity', None, 'test'), ('app_reviews', None, 'train'), ('imdb', None, 'test'), ('rotten_tomatoes', None, 'validation'), ('yelp_review_full', None, 'test'), ('common_gen', None, 'validation'), ('wiki_bio', None, 'val'), ('cnn_dailymail', '3.0.0', 'validation'), ('gigaword', None, 'validation'), ('multi_news', None, 'validation'), ('xsum', None, 'validation'), ('samsum', None, 'validation'), ('ag_news', None, 'test'), ('dbpedia_14', None, 'test'), ('trec', None, 'test')]

# for p3_dataset in configs:
#     data = datasets.load_dataset('bigscience/P3', p3_dataset, cache_dir=cache_dir)

for dataset_name, subset, split in all_datasets:
    print(dataset_name, subset, split)
    dataset = datasets.load_dataset(dataset_name, subset, split=split)

    prompts = DatasetTemplates(dataset, subset)
    prompt_names = prompts.all_template_names
    original_prompts =[n for n in prompt_names if prompts[n].metadata.original_task]
    non_original_prompts = [n for n in prompt_names if not prompts[n].metadata.original_task]

    for pname in prompt_names:
        p3_dataset = f'{dataset_name}_{subset}_{pname}' if subset else f'{dataset_name}_{pname}'
        p3_dataset = datasets.load_dataset('bigscience/P3', p3_dataset, cache_dir=cache_dir)
